# Model for testing

```bash
CUDA_VISIBLE_DEVICES="4,5", python -m vllm.entrypoints.openai.api_server \
--model mistralai/Mistral-7B-Instruct-v0.2 \
--download-dir /secure/chiahsuan/hf_cache/ \
--tensor-parallel-size 2 \
--port 8085
```

# 1 Load packages and define class
== The following two cells can be organized into a separate py file as a module.==

In [14]:
from prompt import system_instruction
from prompt import baseline_prompt_n03 as baseline_prompt
from prompt import initial_predict_prompt_n03 as initial_predict_prompt
from prompt import subsequent_predict_prompt_n03 as subsequent_predict_prompt
from prompt import testing_predict_prompt_n03 as testing_predict_prompt

import openai
from openai import OpenAI
import pandas as pd
from tqdm import tqdm
import time
import json
from pydantic import BaseModel, Field
from typing import List, Dict, Union
from fuzzywuzzy import fuzz
import re
import ast

## 1.1 Define ChoiceAgent

In [2]:
# define class
class ChoiceAgent:
    """ the simplest agent, which is appropriate for zero-shot prompting
    """
    def __init__(self, client: OpenAI, model: str, 
                 prompt_template: str, choices: dict, label: str) -> None:
        self.client = client
        self.model = model
        self.prompt_template = prompt_template
        self.choices = choices
        self.label = label

    def get_response_from_choices(self, messages: list, temperature:float) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            extra_body={"guided_choice":self.choices},
            temperature = temperature
        )
        return response.choices[0].message.content

    def run(self, dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:

        pbar = tqdm(total=dataset.shape[0])
        for idx, row in dataset.iterrows():
            report = row["text"]
            prompt = self.prompt_template.format(report=report)
            system_prompt = system_instruction+ "\n" + prompt
            messages = [{"role": "user", "content": system_prompt}]
            answer = self.get_response_from_choices(messages, temperature)
            dataset.loc[idx, f"zs_{self.label}_ans_str"] = answer
            pbar.update(1)
        pbar.close()

        return dataset

## 1.2 Define MemoryAgent

In [15]:
class MemoryAgent:
    """ the implementation of memory agent, which learn memory from training set and
    utilize memory as contexts for testing set.
    """
    def __init__(self, client: OpenAI, model: str, 
                 prompt_template_dict: dict[str, str], schema_dict: dict, label: str) -> None:
        self.client = client
        self.model = model
        self.prompt_template_dict = prompt_template_dict
        self.validate_prompt_template()
        self.schema_dict = schema_dict
        self.validate_schema()
        self.memory = ""
        self.label = label

    def validate_prompt_template(self) -> None:
        keys = self.prompt_template_dict.keys()
        initial_prompt_exist = "initialized_prompt" in keys
        learning_prompt_exist = "learning_prompt" in keys
        testing_prompt_exist = "testing_prompt" in keys
        assert True == initial_prompt_exist == learning_prompt_exist == testing_prompt_exist, \
        "You should provide a dict with initialized_prompt, learning_prompt, and testing_prompt as keys."

    def validate_schema(self) -> None:
        keys = self.schema_dict.keys()
        learning_schema_exist = "learning_schema" in keys
        testing_schema_exist = "testing_schema" in keys
        assert True == learning_schema_exist == testing_schema_exist, \
        "You should provide a dict with learning_schema and testing_schema as keys."

    def get_schema_followed_response(self, messages: list, schema:dict, temperature:float) -> Union[Dict, None]:
        num_attempt = 3
        for attempt in range(num_attempt):
            try:
                response = self.client.chat.completions.create(
                    model=self.model,
                    messages=messages,
                    extra_body={"guided_json":schema},
                    temperature = temperature
                )
                return json.loads(response.choices[0].message.content.replace("\\", "\\\\"))
            except json.JSONDecodeError:
                print("Error decoding JSON response")
                return None
            except openai.APITimeoutError:
                if attempt < (num_attempt -1):
                    wait_time = 2 * (attempt + 1)
                    print(f"Request timed out. Retrying in {wait_time} seconds...")
                    time.sleep(wait_time)
                else:
                    print("Max retries reached. Request faild.")
                    return None
        
    def train(self, training_dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:
        pbar = tqdm(total=training_dataset.shape[0])
        parsing_error = 0
        for idx, row in training_dataset.iterrows():

            report = row["text"]

            if self.memory == "":
                prompt = self.prompt_template_dict["initialized_prompt"].format(report=report)
            else:
                prompt = self.prompt_template_dict["learning_prompt"].format(memory=self.memory, report=report)
            
            system_prompt = system_instruction+ "\n" + prompt
            messages = [{"role": "user", "content": system_prompt}]

            json_output = self.get_schema_followed_response(messages, self.schema_dict["learning_schema"], temperature)

            if not json_output:
                parsing_error += 1
                continue
            
            self.memory = json_output['rules']
            training_dataset.loc[idx, f"mem_{self.label}_reasoning"] = json_output['reasoning']
            training_dataset.loc[idx, f"mem_{self.label}_ans_str"] = json_output['predictedStage']
            
            pbar.update(1)
        pbar.close()
        return training_dataset
    
    def test(self, testing_dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:
        pbar = tqdm(total=testing_dataset.shape[0])
        parsing_error = 0
        for idx, row in testing_dataset.iterrows():

            report = row["text"]

            prompt = self.prompt_template_dict["testing_prompt"].format(memory=self.memory, report=report)
            system_prompt = system_instruction+ "\n" + prompt
            messages = [{"role": "user", "content": system_prompt}]

            json_output = self.get_schema_followed_response(messages, self.schema_dict["testing_schema"], temperature)

            if not json_output:
                parsing_error += 1
                continue
            
            testing_dataset.loc[idx, f"mem_{self.label}_reasoning"] = json_output['reasoning']
            testing_dataset.loc[idx, f"mem_{self.label}_ans_str"] = json_output['predictedStage']
            
            pbar.update(1)
        pbar.close()
        return testing_dataset

## 1.3 Define ConditionalMemoryAgent

In [16]:
class ConditionalMemoryAgent(MemoryAgent):
  
  def __init__(self, client: OpenAI, model: str, 
                 prompt_template_dict: dict[str, str], schema_dict: dict, label: str) -> None:
    # inherit all properties and methods from MemoryAgent
    super().__init__(client, model, prompt_template_dict, schema_dict, label)
  
  def is_updated(self, new_memory, threshold):
    old_str = "\n".join(self.memory)
    new_str = "\n".join(new_memory)
    if fuzz.ratio(old_str, new_str) >= threshold : 
        return True # update memory
    else:
        return False
    
  def train(self, training_dataset: pd.DataFrame, temperature: float = 0.0, threshold: float = 80) -> pd.DataFrame:
    # only overide this function because the rest parts are the same
    pbar = tqdm(total=training_dataset.shape[0])
    parsing_error = 0
    num_update = 0
    for idx, row in training_dataset.iterrows():

        report = row["text"]

        if self.memory == "":
            prompt = self.prompt_template_dict["initialized_prompt"].format(report=report)
        else:
            prompt = self.prompt_template_dict["learning_prompt"].format(memory=self.memory, report=report)
        
        system_prompt = system_instruction+ "\n" + prompt
        messages = [{"role": "user", "content": system_prompt}]

        json_output = self.get_schema_followed_response(messages, self.schema_dict["learning_schema"], temperature)

        if not json_output:
            parsing_error += 1
            continue
        
        if self.memory == "":
           self.memory = json_output['rules']
        else:
          current_memory_str = "\n".join(self.memory)
          new_memory_str = "\n".join(json_output['rules'])
          if fuzz.ratio(current_memory_str, new_memory_str) >= threshold :
            self.memory = json_output['rules']
            num_update += 1

        training_dataset.loc[idx, f"cmem_{self.label}_reasoning"] = json_output['reasoning']
        training_dataset.loc[idx, f"cmem_{self.label}_ans_str"] = json_output['predictedStage']
        
        pbar.update(1)
    pbar.close()

    print(parsing_error, num_update)
    
    return training_dataset

  def test(self, testing_dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:
    pbar = tqdm(total=testing_dataset.shape[0])
    parsing_error = 0
    for idx, row in testing_dataset.iterrows():

        report = row["text"]

        prompt = self.prompt_template_dict["testing_prompt"].format(memory=self.memory, report=report)
        system_prompt = system_instruction+ "\n" + prompt
        messages = [{"role": "user", "content": system_prompt}]

        json_output = self.get_schema_followed_response(messages, self.schema_dict["testing_schema"], temperature)

        if not json_output:
            parsing_error += 1
            continue
        
        testing_dataset.loc[idx, f"cmem_{self.label}_reasoning"] = json_output['reasoning']
        testing_dataset.loc[idx, f"cmem_{self.label}_ans_str"] = json_output['predictedStage']
        
        pbar.update(1)
    pbar.close()
    return testing_dataset

## 1.4 To-be-Implemented

In [5]:
class MultiTaskMemoryAgent(MemoryAgent):
    def __init__(self, client: OpenAI, model: str, 
                 prompt_template_dict: dict[str, str], schema_dict: dict) -> None:
        # inherit all properties and methods from MemoryAgent
        super().__init__(client, model, prompt_template_dict, schema_dict)
  
    def train(self, training_dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:
        # overide this function
        pass

    def test(self, testing_dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:
        # overide this function
        pass

class MemoryAgentWithVerifier(MemoryAgent): # inherit from MemoryAgent or MultiTaskMemoryAgent
    def __init__(self, client: OpenAI, model: str, 
                 prompt_template_dict: dict[str, str], schema_dict: dict) -> None:
        # inherit all properties and methods from MemoryAgent
        super().__init__(client, model, prompt_template_dict, schema_dict)
  
    def train(self, training_dataset: pd.DataFrame, temperature: float = 0.0) -> pd.DataFrame:
        # overide this function
        pass

# 2 Demonstration

## 2.1 Initialize ChoiceAgent and use its instance

In [6]:
openai_api_key = "Empty"
openai_api_base = "http://localhost:8085/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

brca_report = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/5_folds_summary/brca_df.csv")
print(len(brca_report))
brca_report = brca_report[brca_report["n"]!=-1]
df_samples = brca_report.iloc[:300, :]
df_testing_samples = brca_report.iloc[300:, :]

# df_samples = brca_report.sample(5)
# df_testing_samples = brca_report.sample(30)

print(len(brca_report))
print(len(df_samples))
print(len(df_testing_samples))

1031
800
300
500


In [7]:
# zs_agent = ChoiceAgent(client=client, model="mistralai/Mistral-7B-Instruct-v0.2",
#                  prompt_template=baseline_prompt,
#                  choices=["T1", "T2", "T3", "T4"])

zs_agent = ChoiceAgent(client=client, model="mistralai/Mistral-7B-Instruct-v0.2",
                 prompt_template=baseline_prompt,
                 choices=["N0", "N1", "N2", "N3"], label = "n")

In [8]:
zs_agent.run(df_testing_samples)

  0%|          | 0/500 [00:00<?, ?it/s]/tmp/ipykernel_2546187/660823855.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset.loc[idx, f"zs_{self.label}_ans_str"] = answer
100%|██████████| 500/500 [01:47<00:00,  4.64it/s]


,Unnamed: 0,patient_filename,t,text,type,n,zs_n_ans_str
409,854,TCGA-AR-A1AK.F4DBFC71-9CD0-41C3-9F20-77FA0605D4A8,0,"Final diagnosis. Breast. right, simple mastect...",BRCA,0,N0
410,855,TCGA-AR-A1AL.E3D44C6B-4497-4212-8CA1-AC9444B4CC54,2,"Final Diagnosis. Breast, left, mastectomy: Inf...",BRCA,1,N1
411,856,TCGA-AR-A1AM.10C43124-3C21-443C-8268-23BF8A4D317E,2,"; Final Diagnosis. Breast, right, simple maste...",BRCA,1,N1
412,857,TCGA-AR-A1AN.DA251274-7966-4F48-A02F-393ABBDAB4E9,1,"Final Diagnosis. Breast, right, 'wide local ex...",BRCA,0,N0
413,858,TCGA-AR-A1AO.37EF1828-0A53-4358-97C7-A7849EC653AF,0,"Final diagnosis. Breast, left wide local excis...",BRCA,1,N1
...,...,...,...,...,...,...,...
1023,1509,TCGA-UU-A93S.821ED144-DF12-4E49-ADC7-27FA5E422B83,3,Sex: F. Account: Date Collected: Date Received...,BRCA,3,N3
1024,1510,TCGA-V7-A7HQ.82B46073-10CE-4D05-9421-6308E3FAD1AD,0,SurgicalFinal Report. l'emporary. orderediby. ...,BRCA,2,N2
1028,1514,TCGA-XX-A89A.5D85E578-64B4-4238-922E-802B8ED87800,2,Research Gross Description. Research Dx. Left ...,BRCA,0,N0
1029,1515,TCGA-Z7-A8R5.7726F7AA-88A8-4DD6-B322-6FC68893E0D2,2,ADDENDUM. Addendum #1. Entered. BREAST CANCER ...,BRCA,1,N1


## 2.2 Initialize MemoryAgent and use its instance

In [19]:
class TrainingResponse(BaseModel):
    predictedStage: str = Field(description="predicted cancer stage")
    reasoning: str = Field(description="reasoning to support predicted cancer stage") 
    rules: List[str] = Field(description="list of rules") 
training_schema = TrainingResponse.model_json_schema()

class TestingResponse(BaseModel):
    predictedStage: str = Field(description="predicted cancer stage")
    reasoning: str = Field(description="reasoning to support predicted cancer stage") 
testing_schema = TestingResponse.model_json_schema()

In [20]:
memory_agent = MemoryAgent(client=client, model="mistralai/Mistral-7B-Instruct-v0.2",
                           prompt_template_dict={"initialized_prompt":initial_predict_prompt,
                                                 "learning_prompt":subsequent_predict_prompt,
                                                 "testing_prompt":testing_predict_prompt},
                           schema_dict={"learning_schema":training_schema,
                                        "testing_schema":testing_schema},
                                        label = "n")

In [21]:
memory_agent.train(df_samples)

Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.


Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.


Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.
Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.
Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.
Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.
Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.


Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.
Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...
Max retries reached. Request faild.
Request timed out. Retrying in 2 seconds...
Request timed out. Retrying in 4 seconds...


KeyboardInterrupt: 

In [ ]:
memory_agent.memory

In [ ]:
print(len(memory_agent.memory))

In [ ]:
memory_agent.test(df_testing_samples)

## 2.3 Initialize ConditionalMemoryAgent and use its instance

In [ ]:
conditional_memory_agent = ConditionalMemoryAgent(client=client, model="mistralai/Mistral-7B-Instruct-v0.2",
                           prompt_template_dict={"initialized_prompt":initial_predict_prompt,
                                                 "learning_prompt":subsequent_predict_prompt,
                                                 "testing_prompt":testing_predict_prompt},
                           schema_dict={"learning_schema":training_schema,
                                        "testing_schema":testing_schema},
                                        label = "n")

In [ ]:
conditional_memory_agent.train(df_samples)

In [ ]:
conditional_memory_agent.memory

In [ ]:
len(conditional_memory_agent.memory)

In [ ]:
conditional_memory_agent.test(df_testing_samples)

In [ ]:
df_samples.to_csv("df_training.csv", index=False)
df_testing_samples.to_csv("df_testing.csv", index=False)

In [ ]:
# 300 - 500

## 3 Experiemntal Plan

In [ ]:
# run T task only
T_memory_agent = MemoryAgent(client=client, model="mistralai/Mistral-7B-Instruct-v0.2",
                           prompt_template_dict={"initialized_prompt":initial_predict_prompt,
                                                 "learning_prompt":subsequent_predict_prompt,
                                                 "testing_prompt":testing_predict_prompt},
                           schema_dict={"learning_schema":training_schema,
                                        "testing_schema":testing_schema})

# run N task only
N_memory_agent = MemoryAgent(client=client, model=...,
                           prompt_template_dict=...,
                           schema_dict=...)

# run T and N task simultaneously
TN_memory_agent = MultiTaskMemoryAgent(client=client, model=...,
                           prompt_template_dict=...,
                           schema_dict=...)

```python
""" the following pseudo codes are for checking number of instances are sufficient
"""
def stratified_sampling_function(pd.dataframe, target_column, size):
    pass

def eval():
    pass

def report():
    pass

performances = {}
for size in [5, 10, 20, 25, 30, 35, 40]:
    evaluated_scores = []
    # the process should be evaluated on K different splits and take the average performance
    for i, (train_indexes, test_indexes) in stratified_sampling_function(df): 
        
        # the memory is unique for each split
        N_memory_agent = MemoryAgent(client=client, model=...,
                           prompt_template_dict=...,
                           schema_dict=...)

        df_train = df.iloc[train_indexes,:]
        N_memory_agent.train(df_train)
        df_test = df.iloc[test_indexes,:]
        df_results = N_memory_agent.test(df_test)
        evaluated_scores.append(eval(df_results))

    performances[size] = report(evaluated_scores)
```

# 999 (temporary) Memory saturation test

In [ ]:
def find_memory(file_path):
    with open(file_path, 'r') as file:
        content = file.read()
    matches = re.findall(r" memory: (\[.*?\])", content, re.DOTALL)
    memory_lst = [ast.literal_eval(match) for match in matches]
    return memory_lst

file_path = '/home/yl3427/cylab/selfCorrectionAgent/memory.txt'

memories = find_memory(file_path)

print(len(memories))
print(memories)

In [ ]:
class TestAgent:
    def __init__(self, client: OpenAI, model: str, 
                 prompt_template: str, schema, memory_lst: List[str]) -> None:
        self.client = client
        self.model = model
        self.prompt_template = prompt_template
        self.schema = schema
        self.memory_lst = memory_lst


    def get_schema_followed_response(self, messages: list, schema:dict, temperature:float) -> Union[Dict, None]:
        try:
            response = self.client.chat.completions.create(
                model=self.model,
                messages=messages,
                extra_body={"guided_json":schema},
                temperature = temperature
            )
            return json.loads(response.choices[0].message.content.replace("\\", "\\\\"))
        except json.JSONDecodeError:
            return None

    
    def test(self, testing_dataset: pd.DataFrame, temperature: float = 0.0):
        for mem_idx, memory in enumerate(self.memory_lst):
            print(f"memory index: {mem_idx}")
            parsing_error = 0
            correct_count = 0
            incorrect_count = 0
            for report_idx, row in testing_dataset.iterrows():
                # print(report_idx)
                report = row["text"]
                label = row["t"]

                prompt = self.prompt_template.format(memory=memory, report=report)
                system_prompt = system_instruction+ "\n" + prompt
                messages = [{"role": "user", "content": system_prompt}]

                json_output = self.get_schema_followed_response(messages, self.schema, temperature)

                if not json_output:
                    parsing_error += 1
                    continue
                
                if f"T{label+1}" == json_output["predictedStage"]:
                    correct_count += 1
                else:
                    incorrect_count +=1

            print(f"\tcorrect: {correct_count}")
            print(f"\twrong: {incorrect_count}")
            print(f"\tparsing error: {parsing_error}")
            print()

        

In [ ]:
openai_api_key = "Empty"
openai_api_base = "http://localhost:8000/v1"

client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)

brca_report = pd.read_csv("/secure/shared_data/rag_tnm_results/summary/5_folds_summary/brca_df.csv")
df_testing = brca_report.iloc[500:600, :]

In [ ]:
len(df_testing)

In [ ]:
class TestingResponse(BaseModel):
    predictedStage: str = Field(description="predicted cancer stage")
    reasoning: str = Field(description="reasoning to support predicted cancer stage") 
testing_schema = TestingResponse.model_json_schema()

memory_agent = TestAgent(client=client, model="mistralai/Mistral-7B-Instruct-v0.2",
                           prompt_template = testing_predict_prompt,
                           schema = testing_schema,memory_lst = memories)
memory_agent.test(df_testing)